Build a structural feature matrix (n x m) from a people graph edge list.

Input CSV: exactly two columns representing an undirected edge between person A and person B.
- By default, the script assumes the CSV HAS a header row with column names.
- Use --no-header if your CSV has no header. Then columns are referenced by zero-based indices.
- Self-loops are removed; multiple edges are collapsed (unweighted, simple graph).

Output CSV: rows = people (node IDs), columns = structural features.

Fast features (always on):
- degree
- clustering_coef
- triangle_count
- k_core
- avg_neighbor_degree
- ego_edges
- ego_density
- pagerank

Optional slower features (enable with --heavy):
- eigenvector_centrality
- betweenness_centrality
- constraint (Burt)
- effective_size (Burt)

Usage:
    python build_graph_features.py --input edges.csv --output features.csv \
        --src-col PersonA --dst-col PersonB
    # or for no-header CSVs:
    python build_graph_features.py --input edges.csv --output features.csv --no-header

Notes:
- For large graphs (e.g., > 1M edges), prefer running without --heavy first.
- All features are structure-only (no attributes), computed on an undirected simple graph.

In [13]:
import pandas as pd
import networkx as nx
import numpy as np
from typing import Tuple, Any
import re
from typing import Optional, Sequence

In [9]:
def clean_text(text: str) -> str:
    """
    Remove all characters from the input string except English alphabets (a-z, A-Z).
    Consecutive non-alphabet characters are replaced with a single space.
    
    Args:
        text (str): Input text.
    
    Returns:
        str: Cleaned text containing only English letters and spaces.
    """
    # Keep only letters, replace everything else with space
    cleaned = re.sub(r'[^A-Za-z]+', ' ', text)
    # Strip extra whitespace
    return cleaned.strip()

In [10]:
def read_edges(path: str, has_header: bool, src_col: str, dst_col: str, sep: str) -> pd.DataFrame:
    if has_header:
        df = pd.read_csv(path, sep=sep)
        if src_col is None or dst_col is None:
            # Pick the first two columns by order
            if df.shape[1] < 2:
                raise ValueError("CSV must have at least two columns.")
            src_col = df.columns[0]
            dst_col = df.columns[1]
        # Coerce to string IDs (people names)
        df[src_col] = df[src_col].astype(str).map(clean_text)
        df[dst_col] = df[dst_col].astype(str).map(clean_text)
        df = df[[src_col, dst_col]].rename(columns={src_col: "src", dst_col: "dst"})
    else:
        df = pd.read_csv(path, sep=sep, header=None)
        if df.shape[1] < 2:
            raise ValueError("CSV must have at least two columns.")
        # Use the first two columns
        df = df[[0, 1]].rename(columns={0: "src", 1: "dst"})
        df["src"] = df["src"].astype(str)
        df["dst"] = df["dst"].astype(str)
    return df

def build_graph(df: pd.DataFrame) -> nx.Graph:
    # Undirected, simple graph
    G = nx.Graph()
    # Drop self loops
    df = df[df["src"] != df["dst"]].copy()
    # Add edges (NetworkX deduplicates multiple edges for Graph)
    G.add_edges_from(df[["src", "dst"]].itertuples(index=False, name=None))
    return G

def ego_stats(G: nx.Graph, node: Any) -> Tuple[int, float]:
    d = G.degree(node)
    if d < 2:
        return 0, 0.0
    nbrs = list(G.neighbors(node))
    S = G.subgraph(nbrs)
    e = S.number_of_edges()
    # Density among neighbors only (not including the ego node)
    dens = 2.0 * e / (d * (d - 1))
    return e, dens

def compute_features(G: nx.Graph, heavy: bool) -> pd.DataFrame:
    nodes = list(G.nodes())
    n = len(nodes)
    if n == 0:
        return pd.DataFrame(columns=[])

    # Precompute frequently used dicts
    deg = dict(G.degree())
    clust = nx.clustering(G)  # local clustering coefficient
    tri = nx.triangles(G)     # triangle count per node
    core = nx.core_number(G)  # k-core index
    andeg = nx.average_neighbor_degree(G)

    # Ego stats
    ego_e = {}
    ego_d = {}
    for v in nodes:
        e, d = ego_stats(G, v)
        ego_e[v] = e
        ego_d[v] = d

    # Pagerank (undirected graph is treated as symmetric directed)
    pr = nx.pagerank(G, alpha=0.85, tol=1e-06)

    # Initialize feature dict
    feat = {
        "degree": deg,
        "clustering_coef": clust,
        "triangle_count": tri,
        "k_core": core,
        "avg_neighbor_degree": andeg,
        "ego_edges": ego_e,
        "ego_density": ego_d,
        "pagerank": pr,
    }

    if heavy:
        # Eigenvector centrality
        try:
            ev = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06)
            feat["eigenvector_centrality"] = ev
        except Exception as e:
            print(f"[warn] eigenvector_centrality failed: {e}")

        # Betweenness centrality
        try:
            bc = nx.betweenness_centrality(G, normalized=True)
            feat["betweenness_centrality"] = bc
        except Exception as e:
            print(f"[warn] betweenness_centrality failed: {e}")

        # Structural holes metrics (Burt)
        try:
            eff = nx.effective_size(G)
            feat["effective_size"] = eff
        except Exception as e:
            print(f"[warn] effective_size failed: {e}")
        try:
            con = nx.constraint(G)
            feat["constraint"] = con
        except Exception as e:
            print(f"[warn] constraint failed: {e}")

    # Assemble DataFrame with consistent node ordering
    data = {k: pd.Series(v) for k, v in feat.items()}
    X = pd.DataFrame(data, index=nodes).sort_index()
    # Fill any missing values
    X = X.reindex(sorted(nodes)).fillna(0.0)
    return X


In [14]:
def adjacency_matrix_from_df(
    df: pd.DataFrame,
    nodes: Optional[Sequence[str]] = None,
    weighted: bool = False,
    dtype=float,
) -> pd.DataFrame:
    """
    Build an undirected graph from a two-column edge DataFrame (src, dst)
    and return its n x n adjacency matrix as a pandas DataFrame.

    Args:
        df: DataFrame with columns ["src", "dst"] (strings).
        nodes: Optional explicit node ordering. If None, uses all nodes
               appearing in df, sorted.
        weighted: If True, treat multiple edges as weights (counts).
                  If False, simple 0/1 adjacency.
        dtype: numpy/pandas dtype for the matrix values.

    Returns:
        pd.DataFrame: adjacency matrix with index/columns = node IDs.
    """
    if not {"src", "dst"}.issubset(df.columns):
        raise ValueError('df must have columns "src" and "dst"')

    # Drop self-loops
    edf = df[df["src"] != df["dst"]].copy()

    if weighted:
        # Count parallel edges between same pair
        edf["w"] = 1
        wdf = (
            edf.groupby(["src", "dst"], as_index=False)["w"]
            .sum()
        )
        G = nx.Graph()
        G.add_weighted_edges_from(wdf[["src", "dst", "w"]].itertuples(index=False, name=None))
        weight_attr = "weight"
    else:
        # Simple graph (NetworkX deduplicates)
        G = nx.Graph()
        G.add_edges_from(edf[["src", "dst"]].itertuples(index=False, name=None))
        weight_attr = None

    # Choose node ordering
    if nodes is None:
        nodes = sorted(G.nodes())
    else:
        # Ensure nodes exist in graph (include isolated if provided)
        missing = set(nodes) - set(G.nodes())
        if missing:
            G.add_nodes_from(missing)

    # Build adjacency as DataFrame
    A = nx.to_numpy_array(G, nodelist=list(nodes), dtype=dtype, weight=weight_attr)
    A_df = pd.DataFrame(A, index=nodes, columns=nodes)
    return A_df


In [11]:

sep=","
no_header=True
heavy=True

df = read_edges("linked_In_endoresement_combined_csv.csv", has_header=True, src_col="main_name", dst_col="names", sep=sep)
G = build_graph(df)
X = compute_features(G, heavy=heavy)

# Save, with node names as first column
X.index.name = "person"
X.to_csv("feature_matrix.csv")

print(f"Done. Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
print(f"Wrote features to: .")
print("Columns:", ", ".join(X.columns))

Done. Nodes: 3661, Edges: 3753
Wrote features to: .
Columns: degree, clustering_coef, triangle_count, k_core, avg_neighbor_degree, ego_edges, ego_density, pagerank, eigenvector_centrality, betweenness_centrality, effective_size, constraint


In [22]:
np.save("linkedin_feat.npy",X.values)

In [20]:
adj_matrix = adjacency_matrix_from_df(df)

In [23]:
np.save("linkedin_adj.npy",adj_matrix.values)

In [26]:
all_nodes = adj_matrix.columns.tolist()

In [40]:
len(df["src"].unique())

41

In [32]:
all_influencers = df["src"].unique().tolist()

In [41]:
np.save("linkedin_label.npy",np.array([1 if node in all_influencers else 0 for node in all_nodes]))